# LR w/ClinicalBERT embeddings for baseline performance

In [1]:
import utils
import tensorflow as tf
from transformers import AutoTokenizer
LABELS = ["ABDOMINAL",
        "ADVANCED-CAD",
        "ALCOHOL-ABUSE",
        "ASP-FOR-MI",
        "CREATININE",
        "DIETSUPP-2MOS",
        "DRUG-ABUSE",
        "ENGLISH",
        "HBA1C",
        "KETO-1YR",
        "MAJOR-DIABETES",
        "MAKES-DECISIONS",
        "MI-6MOS"]

c:\Users\nlabu\anaconda3\envs\tfenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\nlabu\anaconda3\envs\tfenv\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
def get_word_embeddings(data):
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

    # Tokenize text
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    clinical_notes = [utils.truncate_clinical_note(utils.tokenize_data(note), 512) for note in list(data['notes'])]
    tokenized_notes = tokenizer(clinical_notes, padding='max_length', max_length=512, truncation=True, return_tensors="tf")

    # Get word embeddings
    word_embeddings = tokenized_notes['input_ids'].numpy()

    return word_embeddings

### Retrieving Data

In [3]:
# Both will be dataframes with a 'notes' column and a column for each label
train_data = utils.get_note_data(LABELS, folder_name='train')
test_data = utils.get_note_data(LABELS, folder_name='test')

### Creating Model

In [4]:
word_embeddings = get_word_embeddings(train_data)
models = {}
for label in LABELS:
    # Define logistic regression model
    logistic_regression_model = tf.keras.Sequential([
        tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(word_embeddings.shape[1],))
    ])

    # Compile the model
    logistic_regression_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    logistic_regression_model.fit(word_embeddings, train_data[label], epochs=20)

    models[label] = logistic_regression_model



Epoch 1/20
7/7 [==============================] - 0s 2ms/step - loss: 3616.4382 - accuracy: 0.5149
Epoch 2/20
7/7 [==============================] - 0s 1ms/step - loss: 3247.9773 - accuracy: 0.5347
Epoch 3/20
7/7 [==============================] - 0s 1ms/step - loss: 2901.4541 - accuracy: 0.5446
Epoch 4/20
7/7 [==============================] - 0s 1ms/step - loss: 2587.2029 - accuracy: 0.5644
Epoch 5/20
7/7 [==============================] - 0s 1ms/step - loss: 2332.3010 - accuracy: 0.5842
Epoch 6/20
7/7 [==============================] - 0s 1ms/step - loss: 2091.3628 - accuracy: 0.6089
Epoch 7/20
7/7 [==============================] - 0s 1ms/step - loss: 1887.5973 - accuracy: 0.6188
Epoch 8/20
7/7 [==============================] - 0s 1ms/step - loss: 1637.7200 - accuracy: 0.6485
Epoch 9/20
7/7 [==============================] - 0s 1ms/step - loss: 1460.7632 - accuracy: 0.6881
Epoch 10/20
7/7 [==============================] - 0s 1ms/step - loss: 1273.3126 - accuracy: 0.6881
Epoch 11/

### Getting Predictions

In [5]:
label_to_predictions = {}
for label, model in models.items():
    print(f"Predicting for model: {label}")
    # Predict
    word_embeddings = get_word_embeddings(test_data)
    label_to_predictions[label] = model.predict(word_embeddings)[:,0].astype(int).tolist()

Predicting for model: ABDOMINAL
3/3 [==============================] - 0s 2ms/step
Predicting for model: ADVANCED-CAD
3/3 [==============================] - 0s 2ms/step
Predicting for model: ALCOHOL-ABUSE
3/3 [==============================] - 0s 2ms/step
Predicting for model: ASP-FOR-MI
3/3 [==============================] - 0s 2ms/step
Predicting for model: CREATININE
3/3 [==============================] - 0s 2ms/step
Predicting for model: DIETSUPP-2MOS
3/3 [==============================] - 0s 2ms/step
Predicting for model: DRUG-ABUSE
3/3 [==============================] - 0s 2ms/step
Predicting for model: ENGLISH
3/3 [==============================] - 0s 2ms/step
Predicting for model: HBA1C
3/3 [==============================] - 0s 3ms/step
Predicting for model: KETO-1YR
3/3 [==============================] - 0s 2ms/step
Predicting for model: MAJOR-DIABETES
3/3 [==============================] - 0s 2ms/step
Predicting for model: MAKES-DECISIONS
3/3 [==============================] 

Save predictions and load predictions here if wanted.

In [6]:
utils.save_preds(label_to_predictions, "LR_predictions")

In [7]:
label_to_predictions = utils.read_preds("LR_predictions")

c:\Users\nlabu\OneDrive - Northeastern University\Documents\GitHub\Final-Project\utils.py:132: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label_to_predictions[row[0]] = list(row[1:])


### Performance:

In [8]:
label_to_micro_f1, overall_f1 = utils.get_f1_scores_for_labels(LABELS, test_data, label_to_predictions)
print('overall-f1:', overall_f1)

Raw f1 for ABDOMINAL 0.5804691762138571
Raw f1 for ADVANCED-CAD 0.5558499511923023
Raw f1 for ALCOHOL-ABUSE 0.4904458598726115
Raw f1 for ASP-FOR-MI 0.5442125237191651
Raw f1 for CREATININE 0.569767441860465
Raw f1 for DIETSUPP-2MOS 0.482736974262398
Raw f1 for DRUG-ABUSE 0.4896551724137931
Raw f1 for ENGLISH 0.43301707779886156
Raw f1 for HBA1C 0.5133660665575559
Raw f1 for KETO-1YR 0.5
Raw f1 for MAJOR-DIABETES 0.6118993135011441
Raw f1 for MAKES-DECISIONS 0.4795321637426901
Raw f1 for MI-6MOS 0.47014925373134325
overall-f1: 0.5170077672973991
